In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import model_selection, preprocessing
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
df_train = pd.read_csv('clean_train.csv')
df_test = pd.read_csv('clean_test.csv')
id_test = df_test.ID
id_train = df_train.ID

In [28]:
y_train = df_train["y"]
x_train = df_train.drop(["y"], axis=1)
x_test = df_test#.drop(["ID"], axis=1)

print x_train.shape, x_test.shape, y_train.shape

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True) 

print x_train.shape, x_test.shape

(4209, 311) (4209, 311) (4209,)
(4209, 311) (4209, 311)


In [4]:
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2020)

In [29]:
params = {  
    "n_estimators": range(80, 121, 10),
    'max_features':np.arange(0.3, 0.8, 0.05),
    'max_depth':range(1, 11),
    'min_samples_split':range(2, 21)#,
#     'oob_score': True
}
    
iters = 100

rfreg = RandomForestRegressor()

In [30]:
gs = RandomizedSearchCV(rfreg, params, n_iter=iters, n_jobs=10, scoring='r2', cv=5)#rmsle_scorer)
gs.fit(x_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=10,
          param_distributions={'n_estimators': [80, 90, 100, 110, 120], 'max_features': array([ 0.3 ,  0.35,  0.4 ,  0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75]), 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='r2', verbose=0)

### gs.cv_results_

In [31]:
df = pd.DataFrame(gs.cv_results_)

In [72]:
df['diff'] = abs(df['mean_test_score'] - df['mean_train_score'])
df['mean'] = df[['mean_test_score', 'mean_train_score']].mean(axis=1)
priority_cols = ['diff', 'mean', 'mean_train_score','mean_test_score']
df = df[priority_cols + list(df.columns.drop(priority_cols))]

subset = df#[(df['mean_test_score'] > 0.55) & (df['diff'] < 0.02)]
# subset.sort(columns=['diff'], inplace=True, ascending=True)
subset.sort(columns=['mean_test_score'], inplace=True, ascending=False)
subset

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,diff,mean,mean_train_score,mean_test_score,mean_fit_time,mean_score_time,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
83,0.035155,0.589619,0.607196,0.572041,3.810162,0.272064,5,0.3,4,120,"{u'n_estimators': 120, u'min_samples_split': 4...",1,0.593178,0.597616,0.576141,0.604651,0.607795,0.599751,0.478849,0.633556,0.604206,0.600406,0.024940,0.008372,0.047865,0.013377
64,0.019877,0.581182,0.591121,0.571243,3.426429,0.233457,4,0.45,12,100,"{u'n_estimators': 100, u'min_samples_split': 1...",2,0.587128,0.580875,0.577073,0.589740,0.605332,0.583344,0.482206,0.618330,0.604445,0.583316,0.022876,0.005615,0.045766,0.013918
98,0.043251,0.592716,0.614341,0.571090,3.515792,0.233826,6,0.3,20,100,"{u'n_estimators': 100, u'min_samples_split': 2...",3,0.587818,0.603259,0.574830,0.613655,0.605227,0.607417,0.483111,0.640034,0.604437,0.607341,0.014018,0.013407,0.045400,0.013269
57,0.059461,0.600701,0.630431,0.570970,3.333292,0.257992,6,0.35,3,80,"{u'n_estimators': 80, u'min_samples_split': 3,...",4,0.588762,0.620697,0.575371,0.629551,0.607140,0.623661,0.478744,0.653599,0.604800,0.624650,0.192673,0.151119,0.047514,0.011929
74,0.038686,0.590313,0.609656,0.570970,4.838702,0.214007,5,0.6,9,100,"{u'n_estimators': 100, u'min_samples_split': 9...",5,0.586900,0.598252,0.574414,0.608864,0.608348,0.603297,0.479317,0.637261,0.605842,0.600607,0.015821,0.011647,0.047482,0.014248
11,0.020898,0.581332,0.591781,0.570883,3.301724,0.216866,4,0.45,6,90,"{u'n_estimators': 90, u'min_samples_split': 6,...",6,0.587347,0.581596,0.576852,0.590241,0.606177,0.584744,0.480178,0.618321,0.603828,0.584002,0.018593,0.021714,0.046603,0.013569
82,0.019636,0.580349,0.590167,0.570531,3.062023,0.185732,4,0.45,18,90,"{u'n_estimators': 90, u'min_samples_split': 18...",7,0.586939,0.579601,0.576877,0.588441,0.603971,0.581163,0.481569,0.618829,0.603267,0.582803,0.013214,0.011954,0.045624,0.014638
29,0.038076,0.589157,0.608195,0.570119,3.476110,0.175313,5,0.5,9,80,"{u'n_estimators': 80, u'min_samples_split': 9,...",8,0.589298,0.599554,0.572047,0.606883,0.605517,0.598756,0.476672,0.634174,0.607031,0.601609,0.017558,0.017775,0.048400,0.013295
67,0.056639,0.598248,0.626567,0.569929,5.510300,0.268260,6,0.45,7,120,"{u'n_estimators': 120, u'min_samples_split': 7...",9,0.588599,0.616649,0.574191,0.627415,0.604514,0.617925,0.477662,0.653327,0.604643,0.617520,0.034833,0.008692,0.047487,0.013941
10,0.039760,0.589742,0.609623,0.569862,6.726943,0.276443,5,0.65,10,120,"{u'n_estimators': 120, u'min_samples_split': 1...",10,0.587211,0.599529,0.573415,0.607400,0.604704,0.601978,0.477916,0.637302,0.606034,0.601902,0.016520,0.013559,0.047505,0.014078


In [55]:
df.ix[64].params

{'max_depth': 4,
 'max_features': 0.44999999999999996,
 'min_samples_split': 12,
 'n_estimators': 100}

In [35]:
gs.best_score_

0.57204148246636366

In [12]:
# model = gs.best_estimator_

model = RandomForestRegressor(max_depth = 4,
                             max_features = 0.45,
                             min_samples_split = 12,
                             n_estimators = 100,
                             random_state=2017)
model.fit(x_train, y_train)
# model.fit(x_tr, y_tr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features=0.45, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=12, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=2017,
           verbose=0, warm_start=False)

In [13]:
cv_scores = cross_val_score(model, x_train, y_train, scoring='r2', cv=10)
print cv_scores
print np.mean(cv_scores)

[ 0.58540311  0.62796238  0.37159825  0.54223854  0.6801262   0.53723575
  0.49599627  0.60669016  0.63746263  0.67003345]
0.575474673819


In [14]:
y_pred_tr = model.predict(x_tr)
print 'Train:', r2_score(y_tr, y_pred_tr)

y_pred_val = model.predict(x_val)
print 'Val:', r2_score(y_val, y_pred_val)
# Train: ('r2', 0.60473081638029402)
# Val: ('r2', 0.56729882111087759)

# Train: ('r2', 0.59019025799407598)
# Val: ('r2', 0.56887638635435567)

# 0.6
# Train: ('r2', 0.58691791141382998)
# Val: ('r2', 0.5681459527865238)

Train: 0.586215345687
Val: 0.577746714302


In [29]:
y_predict = model.predict(x_train)
output = pd.DataFrame({'ID': id_train, 'y_actual': y_train, 'y': y_predict})
print output.shape
output.head()

(4209, 3)


,ID,y,y_actual
0,0,108.831303,130.81
1,6,95.489592,88.53
2,7,77.909271,76.26
3,9,77.713579,80.62
4,13,78.113244,78.02


In [30]:
output.to_csv('train_preds/xgb_rf100_ix83.csv', index=False)

In [17]:
y_predict = model.predict(x_test)
output = pd.DataFrame({'ID': id_test, 'y': y_predict})
output.head()

,ID,y
0,1,78.044349
1,2,95.922860
2,3,78.160497
3,4,77.863431
4,5,112.812853


In [18]:
output.to_csv('submissions/xgb_rf100_ix83.csv', index=False)

In [19]:
output.shape

(4209, 2)

In [75]:
sample = pd.read_csv('sample_submission.csv')
sample.shape

(4209, 2)

In [76]:
sample.head()

,ID,y
0,1,100.669318
1,2,100.669318
2,3,100.669318
3,4,100.669318
4,5,100.669318


In [24]:
import operator
scores = dict(zip(x_test.columns, model.feature_importances_))
sorted_x = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
sorted_x

[('X314', 0.25891840430087693),
 ('X261', 0.22200047359067215),
 ('X127', 0.14398719999654172),
 ('X232', 0.051981884786819768),
 ('X315', 0.04753962968952246),
 ('X118', 0.035456726085395192),
 ('X263', 0.034863123468664652),
 ('X279', 0.031471305524763717),
 ('X29', 0.031034398239710076),
 ('X119', 0.030476622083411321),
 ('X115', 0.01550131263888648),
 ('X136', 0.0095880503882484654),
 ('X76', 0.0081617014394381928),
 ('X54', 0.0070419378620745912),
 ('X0', 0.0050327767188978736),
 ('X313', 0.0049646997195160148),
 ('X311', 0.003815964847045742),
 ('ID', 0.0030933947336262957),
 ('X162', 0.0030355308936381481),
 ('X316', 0.0029818834645549599),
 ('Unnamed: 0', 0.0029711404543705678),
 ('X47', 0.0026873467968124031),
 ('X328', 0.0025552505476744976),
 ('X8', 0.002401329639776557),
 ('X276', 0.0021896731772667421),
 ('X179', 0.002166508017871752),
 ('X80', 0.0017839343316685991),
 ('X117', 0.0016624198861897894),
 ('X264', 0.0016154616482005387),
 ('X265', 0.0013621568296935422),
 ('X